# Compute genes coverage

In [2]:
import os, time, sys
import pandas as pd
import numpy as np

sys.path.append('Code/')
from utility import clean_ds_store

clean_ds_store()

Please pay attention to the following facts:
 * Not all genomes have the "product" attribute;
 * Not all strains present in the dataset have the corresponding .gff file in the gff folder;
 * There are some special cases such as 1150460.13.fna. 

In [9]:
def find_file(file_name):
    """
        Obtain the path of a .gff file.
    """
        
    folder_paths = ['Data/BacDive/gff/Patric/', 'Data/BacDive/gff/ncbi/Decompress/']

    for folder_path in folder_paths:
        for file in os.listdir(folder_path):
            if file.startswith(file_name):
                return os.path.join(folder_path, file)
    
    return None



def get_files_name():   
    """
        Obtain the names of each .wri file.
    """
    
    filemap = dict()
    files = [f for f in os.listdir('Data/BacDive/AFLP_perl') if os.path.isfile('Data/BacDive/AFLP_perl/'+f)]
    
    for f in files:
        if f == '.DS_Store':
            continue
        
        if 'GCA' in f:
            filemap[f[:f.index('.')]] = f
        else:
            filemap[f.replace('.wri','')] = f
            
    return filemap



def get_genomes(file_path):
    """
        Given the path of a .wri file, returns a dataframe of each genomes of the strain.
    """
    
    column_names = ['Sequence', 'Database', 'type', "5'", "3'", 'idk', 'helix', 'idk2', 'info']
    genome = pd.read_csv(file_path, sep='\t', comment='#', header=None, names=column_names)
    genome = genome[genome['type'] == 'CDS']
    
    tmp = genome['info'].str.split(';', expand=True)        
    ids = tmp.apply(lambda x: next((val.split('=')[1] for val in x if 'ID=' in val), None), axis=1).to_frame()
    products = tmp.apply(lambda x: next((val.split('=')[1] for val in x if 'product=' in val), "N/A") if any('product=' in val for val in x if isinstance(val, str)) else "N/A", axis=1).to_frame()
    locus_tags = tmp.apply(lambda x: next((val.split('=')[1] for val in x if 'locus_tag=' in val), "N/A") if any('locus_tag=' in val for val in x if isinstance(val, str)) else "N/A", axis=1).to_frame()
    
    tmp = pd.concat([ids, products, locus_tags], axis=1)
    tmp.columns = ['ID', 'Product', 'Locus tag']

    genome.drop(columns='info', inplace=True)
    genome = pd.concat([genome, tmp], axis=1)

    return genome

        
    
def get_genome_coverage(strain, filemap, aflp_min=50, aflp_max=500, feature_importances=None):   
    """
        Calculate the coverage of the DNA for each strain, determine the location of the genes, 
        and assess whether they are completely, partially, or not covered by AFLP. 
        Then, calculate the importance of each gene based on the importance of the AFLP fragment where the gene is located.
    """
    
    DNA_coverages = dict()
    DNA_importances = dict()
    features = []
    sequence = None
    
    genome_coverage = {
        'Sequence': [],
        'Sequence length': [],
        'Start': [],
        'End': [],
        'Helix': [],
        'Locus tag': [],
        'Product': [],
        'Type': [],
        'Captured nucleotides': [],
        'Coverage over genome length': [],
        'Importance': [],
    }
        
    # Open the strain.wri file and read the AFLP.
    # This files contains only information about Sequences, coordinates of each fragments and their lengths.
    if strain in filemap.keys():
        with open('Data/BacDive/AFLP_perl/' + filemap[strain], 'r') as file:            
            for line in file:
                row = line.strip().split('\t')

                if len(row) == 9:
                    frag_length = int(row[7])

                    if (frag_length >= aflp_min) and (frag_length <= aflp_max):
                        sequence = row[1]

                        if sequence not in DNA_coverages:
                            DNA_coverages[sequence] = [False for i in range(int(row[2]))]
                            DNA_importances[sequence] = [0 for i in range(int(row[2]))]
                        
                        start = int(row[4])
                        end = int(row[6])
                        start = min(start, end)
                        importance = feature_importances.loc[:, int(frag_length)].iloc[0]
                        max_importance = max(DNA_importances[sequence][start:start+frag_length])                        

                        if max_importance < importance:
                            for i in range(start, start+frag_length):
                                DNA_coverages[sequence][i] = True
                                DNA_importances[sequence][i] = importance
                        else:
                            for i in range(start, start+frag_length):
                                if DNA_importances[sequence][i] < importance:
                                    DNA_coverages[sequence][i] = True
                                    DNA_importances[sequence][i] = importance

    
    # Find the relative .gff file
    # This files gives information about where the genes are.
    path = find_file(file_name=strain)

    if path is None:
        return f'file {strain} not found.'
    
    genomes = get_genomes(path)
        
        
    # sequence variable could be null due to .wri files which have sequence length less or greather then aflp_min and aflp_max.
    if sequence is not None:
        
        for index, row in genomes.iterrows():

            sequence_id = row.loc['Sequence']

            if 'accn|' in sequence_id:
                sequence_id = sequence_id[5::]

            if sequence_id not in DNA_coverages.keys():
                continue

            sequence_length = len(DNA_coverages[sequence_id])
            start = min(row.loc["5'"] - 1, row.loc["3'"] - 1)
            end = max(row.loc["5'"] - 1, row.loc["3'"] - 1)
            helix = row.loc['helix']            
            product = row.loc['Product']
            locus_tag = row.loc['Locus tag']            
            genome_length = end - start

            coverage = sum(DNA_coverages[sequence_id][start:end])
            coverage2 = coverage / genome_length
            genome_importance = np.mean(DNA_importances[sequence_id][start:end])

            if coverage2 == 0:
                classification = 'N'
            elif coverage2 < 1:
                classification = 'P'
            else:
                classification = 'T'                

            genome_coverage['Sequence'].append(sequence_id)
            genome_coverage['Sequence length'].append(sequence_length)
            genome_coverage['Start'].append(start+1)
            genome_coverage['End'].append(end+1)
            genome_coverage['Helix'].append(helix)
            genome_coverage['Locus tag'].append(locus_tag)
            genome_coverage['Product'].append(product)
            genome_coverage['Type'].append(classification)
            genome_coverage['Captured nucleotides'].append(f'{coverage}/{genome_length}')
            genome_coverage['Coverage over genome length'].append(coverage2)
            genome_coverage['Importance'].append(genome_importance)
                                
    return genome_coverage



Execution of the programs.

In [8]:
coverages = None
filemap = get_files_name()

strains = filemap.keys()
# strains = ['0'] # Local test
# strains = ['1255.14']
# strains = ['GCA_019655915'] # non presente nella cartella gff
# strains = ['GCA_000008565'] # strain non presente nella cartella AFLP_perl

print('Info: all result will be saved in Result/Coverages/\n')


if os.path.exists('./Result/feature_importances.xlsx'):
    feature_importances = pd.read_excel('./Result/feature_importances.xlsx').set_index('Unnamed: 0').rename_axis(['Carbohydrates'])

for carbohydrate, line in feature_importances.iterrows():
    
    start_time = time.time()
    
    folder = f'Result/Coverages/Gene_Importances/{carbohydrate}/'
    if not os.path.exists(f'Result/Coverages/Gene_Importances/{carbohydrate}/'):
        os.makedirs(folder)
    
    for strain in strains:

        genome_coverage = get_genome_coverage(strain, filemap, feature_importances=line.to_frame().T)
        
        if isinstance(genome_coverage, str):
            # print(f'** Error: {genome_coverage}')
            pass
        else:
            genome_coverage = pd.DataFrame(genome_coverage).sort_values(by='Importance', ascending=False)
            genome_coverage.to_excel(os.path.join(folder, f'{strain}.xlsx'), index=False)
    
    end_time = time.time()
    print(f"Execution time for {carbohydrate}: {end_time - start_time} seconds.")



Info: all result will be saved in Result/Coverages/

Execution time for D-RIBose: 714.3540711402893 seconds.
Execution time for N-AcetylGlucosamine: 773.6110079288483 seconds.
Execution time for SALicin: 774.4000940322876 seconds.
Execution time for D-CELlobiose: 757.5753448009491 seconds.
Execution time for D-LACtose (bovine origin): 817.5171251296997 seconds.
Execution time for D-MELibiose: 734.8802661895752 seconds.
Execution time for D-SACcharose (sucrose): 734.4532721042633 seconds.
Execution time for D-TREhalose: 753.1838119029999 seconds.
Execution time for Methyl-αD-Glucopyranoside: 731.5646698474884 seconds.
Execution time for D-TURanose: 794.804967880249 seconds.
Execution time for D-MANnitol: 800.454832315445 seconds.
Execution time for AMYgdalin: 719.4087779521942 seconds.
Execution time for D-XYLose: 681.100686788559 seconds.
Execution time for D-RAFfinose: 655.4781801700592 seconds.
Execution time for GENtiobiose: 664.7213449478149 seconds.
Execution time for L-ARAbinose:

### Example

In [ ]:
coverages = None
filemap = get_files_name()

strains = filemap.keys()

genome_coverage2 = get_genome_coverage('1423786.4', filemap, feature_importances=feature_importances)
genome_coverage2 = pd.DataFrame(genome_coverage2).set_index('Sequence').sort_values(by='Coverage over genome length', ascending=False)
display(genome_coverage2)